In [1]:
import pandas as pd
df = pd.read_csv("../data/sentiment140.csv", header=None, error_bad_lines=False, encoding='latin1')

df = df[[0, 5]]
df.columns = ["label", "text"]
df['label'] = df.label.factorize()[0]

In [3]:
from sklearn.utils import shuffle
df = shuffle(df, random_state=42)
df_sample = df.iloc[:]
df_sample['text'] = df_sample['text'].apply(lambda x: x.lower())

In [4]:
import numpy as np
def encoding(texts, max_len=None):
    tx2chs = [list(text) for text in texts]
    if max_len is None:
        max_len = max([len(chs) for chs in tx2chs])
    vectors = [list(map(ord, chs))+[0 for _ in range(max_len-len(chs))] for chs in tx2chs]
    max_features = max([max(x) for x in vectors])
    return np.array(vectors), max_len, max_features

X, max_len, max_features = encoding(df_sample['text'])

In [12]:
max_len, max_features

(374, 251)

In [5]:
from keras.utils import to_categorical
y = to_categorical(df_sample['label'])

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
from keras.layers import Input, Dense, Embedding, Flatten
from keras.layers import SpatialDropout1D
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)

In [8]:
model = Sequential()
model.add(Embedding(max_features+1, 150, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(Conv1D(32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [9]:
epochs = 30
batch_size = 2000
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size)

Train on 900000 samples, validate on 300000 samples
Epoch 1/30
900000/900000 [==============================] - 48s 53us/step - loss: 0.6249 - acc: 0.6506 - val_loss: 0.5983 - val_acc: 0.6864
Epoch 2/30
900000/900000 [==============================] - 45s 50us/step - loss: 0.5744 - acc: 0.7046 - val_loss: 0.5418 - val_acc: 0.7357
Epoch 3/30
900000/900000 [==============================] - 45s 50us/step - loss: 0.5458 - acc: 0.7285 - val_loss: 0.5419 - val_acc: 0.7391
Epoch 4/30
900000/900000 [==============================] - 45s 50us/step - loss: 0.5286 - acc: 0.7414 - val_loss: 0.5133 - val_acc: 0.7499
Epoch 5/30
900000/900000 [==============================] - 45s 50us/step - loss: 0.5163 - acc: 0.7496 - val_loss: 0.5247 - val_acc: 0.7400
Epoch 6/30
900000/900000 [==============================] - 45s 50us/step - loss: 0.5070 - acc: 0.7568 - val_loss: 0.5027 - val_acc: 0.7554
Epoch 7/30
900000/900000 [==============================] - 45s 51us/step - loss: 0.5000 - acc: 0.7614 - val

In [14]:
y_preds = model.predict(X_test)
y_preds = np.argmax(y_preds, axis=1)
y_true = np.argmax(y_test, axis=1)

In [15]:
from sklearn.metrics import roc_auc_score, accuracy_score
roc_auc_score(y_true, y_preds), accuracy_score(y_true, y_preds)

(0.7569090255293192, 0.7568025)

In [16]:
model.save("model_2018-08-27-10:22.h5")

In [17]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size)

Train on 900000 samples, validate on 300000 samples
Epoch 1/3
900000/900000 [==============================] - 42s 47us/step - loss: 0.5056 - acc: 0.7582 - val_loss: 0.4882 - val_acc: 0.7681
Epoch 2/3
900000/900000 [==============================] - 42s 47us/step - loss: 0.4958 - acc: 0.7650 - val_loss: 0.4926 - val_acc: 0.7724
Epoch 3/3
900000/900000 [==============================] - 42s 47us/step - loss: 0.4882 - acc: 0.7704 - val_loss: 0.4750 - val_acc: 0.7796


In [18]:
y_preds = model.predict(X_test)
y_preds = np.argmax(y_preds, axis=1)

In [19]:
roc_auc_score(y_true, y_preds), accuracy_score(y_true, y_preds)

(0.7799538228919617, 0.7799725)

In [20]:
model.save("model_2018-08-27-10:25.h5")

In [7]:
from keras.models import load_model
model = load_model("model_2018-08-27-10:25.h5")

In [10]:
epochs = 10
batch_size = 2000
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size)

Train on 900000 samples, validate on 300000 samples
Epoch 1/10
900000/900000 [==============================] - 46s 51us/step - loss: 0.4839 - acc: 0.7732 - val_loss: 0.4758 - val_acc: 0.7753
Epoch 2/10
900000/900000 [==============================] - 42s 46us/step - loss: 0.4799 - acc: 0.7756 - val_loss: 0.4652 - val_acc: 0.7847
Epoch 3/10
900000/900000 [==============================] - 42s 46us/step - loss: 0.4769 - acc: 0.7775 - val_loss: 0.4901 - val_acc: 0.7751
Epoch 4/10
900000/900000 [==============================] - 42s 46us/step - loss: 0.4742 - acc: 0.7789 - val_loss: 0.4671 - val_acc: 0.7859
Epoch 5/10
900000/900000 [==============================] - 42s 46us/step - loss: 0.4718 - acc: 0.7810 - val_loss: 0.4618 - val_acc: 0.7881
Epoch 6/10
900000/900000 [==============================] - 42s 46us/step - loss: 0.4698 - acc: 0.7820 - val_loss: 0.4626 - val_acc: 0.7893
Epoch 7/10
900000/900000 [==============================] - 42s 46us/step - loss: 0.4678 - acc: 0.7833 - val

In [12]:
y_preds = model.predict(X_test)
y_preds = np.argmax(y_preds, axis=1)

In [13]:
from sklearn.metrics import roc_auc_score, accuracy_score
y_true = np.argmax(y_test, axis=1)
roc_auc_score(y_true, y_preds), accuracy_score(y_true, y_preds)

(0.7893832450051664, 0.7893125)

In [14]:
model.save("model_2018-08-27-10:46.h5")

In [15]:
epochs = 10
batch_size = 2000
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size)

Train on 900000 samples, validate on 300000 samples
Epoch 1/10
900000/900000 [==============================] - 44s 49us/step - loss: 0.4614 - acc: 0.7878 - val_loss: 0.4607 - val_acc: 0.7915
Epoch 2/10
900000/900000 [==============================] - 45s 50us/step - loss: 0.4605 - acc: 0.7883 - val_loss: 0.4632 - val_acc: 0.7913
Epoch 3/10
900000/900000 [==============================] - 45s 50us/step - loss: 0.4593 - acc: 0.7887 - val_loss: 0.4548 - val_acc: 0.7900
Epoch 4/10
900000/900000 [==============================] - 45s 50us/step - loss: 0.4584 - acc: 0.7893 - val_loss: 0.4527 - val_acc: 0.7918
Epoch 5/10
900000/900000 [==============================] - 45s 50us/step - loss: 0.4570 - acc: 0.7901 - val_loss: 0.4530 - val_acc: 0.7948
Epoch 6/10
900000/900000 [==============================] - 45s 50us/step - loss: 0.4563 - acc: 0.7909 - val_loss: 0.4549 - val_acc: 0.7945
Epoch 7/10
900000/900000 [==============================] - 45s 50us/step - loss: 0.4551 - acc: 0.7912 - val

In [16]:
y_preds = model.predict(X_test)
y_preds = np.argmax(y_preds, axis=1)

In [17]:
roc_auc_score(y_true, y_preds), accuracy_score(y_true, y_preds)

(0.796649954004895, 0.7966525)

In [18]:
model.save("model_2018-08-27-10:55.h5")

In [19]:
epochs = 10
batch_size = 2000
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size)

Train on 900000 samples, validate on 300000 samples
Epoch 1/10
900000/900000 [==============================] - 44s 49us/step - loss: 0.4522 - acc: 0.7936 - val_loss: 0.4514 - val_acc: 0.7963
Epoch 2/10
900000/900000 [==============================] - 45s 50us/step - loss: 0.4513 - acc: 0.7938 - val_loss: 0.4614 - val_acc: 0.7860
Epoch 3/10
900000/900000 [==============================] - 45s 50us/step - loss: 0.4511 - acc: 0.7939 - val_loss: 0.4506 - val_acc: 0.7932
Epoch 4/10
900000/900000 [==============================] - 45s 50us/step - loss: 0.4504 - acc: 0.7941 - val_loss: 0.4500 - val_acc: 0.7941
Epoch 5/10
900000/900000 [==============================] - 45s 50us/step - loss: 0.4499 - acc: 0.7948 - val_loss: 0.4550 - val_acc: 0.7956
Epoch 6/10
900000/900000 [==============================] - 45s 50us/step - loss: 0.4494 - acc: 0.7952 - val_loss: 0.4482 - val_acc: 0.7975
Epoch 7/10
900000/900000 [==============================] - 45s 50us/step - loss: 0.4488 - acc: 0.7951 - val

In [20]:
y_preds = model.predict(X_test)
y_preds = np.argmax(y_preds, axis=1)
roc_auc_score(y_true, y_preds), accuracy_score(y_true, y_preds)

(0.7973590883686956, 0.7974025)

In [21]:
model.save("model_2018-08-27-11:15.h5")

In [26]:
examples = ["Fuck you asshole.", "I am happy", "very scared...", "why do people hate Trump?"]
example_enc,_,_ = encoding(examples, max_len)
model.predict(example_enc)

array([[0.7620187 , 0.23798132],
       [0.07904999, 0.92095006],
       [0.9092788 , 0.09072125],
       [0.9782016 , 0.02179843]], dtype=float32)

(340700, 10)